In [1]:
import os
import math
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torch.nn as nn
from typing import List
from torch import Tensor
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import Tensor


In [2]:
from dataclasses import dataclass, field

@dataclass
class Config:
    PLAYERS_SIZE: int = 0
    CONTEXT_LEN: int = 16  ## predict the 21st match by the 20
    PERFORMANCE_INPUT_DIM = 23
    PERFORMANCE_EMBD_DIM: int = 128
    
    PLAYER_INPUT_DIM: int = 25   ## univpalyer
    MATCH_INPUT_EMBD: int = 14   ## match_info
    NUM_EPOCHS: int = 100
    LEARNING_RATE: float = 1e-3
    BATCH_SIZE: int = 32
    DEVICE: str = field(default_factory=lambda: "cuda" if torch.cuda.is_available() else "cpu")
    # MODEL_SAVE_PATH: str = "/kaggle/working"
    # BASE_DIR: str = '/kaggle/input/flickr8k/'
    # CROSSATT_NUM_HEADS: int = 8
    CLS_INIT_STD: float = 0.02    ## <CLS> token initialized with std 0.02 from the mean=0
    TEST_DATASET_SIZE: int = 180
    IDLE_DEVICE: str = 'cpu'
    ACCUMULATION_STEPS = 4


In [3]:
config = Config()

## Defination of **Losses**

# Data

In [4]:
import os
import numpy as np
import pandas as pd
from torch.utils.data import Dataset

class PlayerMatchDataset(Dataset):
    """
    Dataset for autoregressive next-match prediction.

    For each sample:
      1. Given a player_id, retrieve universal player features from universal_player.csv.
         (The 'player_id' and 'cricinfo_id' columns are dropped from the input features.)
      2. Retrieve all matches for that player from a directory of player match files
         (each file is named "{player_id}.csv" and contains the matches for that player,
         already sorted by date).
         The performance columns include:
           batting_position, runs, balls, fours, sixes, strike_rate, overs, total_balls, dots,
           maidens, conceded, fours_conceded, sixes_conceded, wickets, LBW, Bowled, noballs,
           wides, economy_rate, catches, stumping, direct_hit, indirect_hit, strike_rate_fp,
           batting_fp, bowling_fp, fielding_fp, total_fp.
         We drop 'match_id' and the fantasy-breakdown columns
         ('strike_rate_fp', 'batting_fp', 'bowling_fp', 'fielding_fp', 'total_fp') when forming the input vector.
      3. Randomly sample a contiguous window of (context_len + 1) matches for this player.
         The first context_len matches serve as input and matches 2 ... context_len+1 yield the target fantasy scores.
      4. For each match in the window:
         - Load the corresponding match players CSV from the match_players folder (file: '{match_id}.csv').
         - Determine the player's team for that match and then separate player_ids into:
              team1_ids: those belonging to the same team as the player,
              team2_ids: those belonging to the other team.
         - Retrieve their universal features from universal_player.csv.
         - Retrieve match info from a single match_info CSV (by matching on match_id).
      5. Return a dictionary containing:
         - 'player_id': the player's id.
         - 'univ_features': the player's universal features.
         - 'context_matches': a numpy array of performance features for the context matches.
         - 'target_scores': a numpy array of target fantasy scores (for matches 2 ... context_len+1).
         - 'team1_players': list (per match) of team1 players' universal features.
         - 'team2_players': list (per match) of team2 players' universal features.
         - 'match_info': list (per match) of match info dictionaries.
    """

    def __init__(self, universal_player_csv, player_matches_dir, match_players_dir, match_info_csv, context_len=25, transform=None):
        """
        Args:
            universal_player_csv (str): Path to universal_player.csv.
            player_matches_dir (str): Directory containing CSV files for each player's matches (named '{player_id}.csv').
            match_players_dir (str): Directory containing match_players CSV files.
            match_info_csv (str): Path to the CSV file containing match info for all matches.
            context_len (int): Number of context matches to use as input 
                                (target will be matches 2 ... context_len+1).
            transform (callable, optional): Optional transform to be applied on a sample.
        """
        self.context_len = context_len
        self.transform = transform

        # Load universal player features and set player_id as index.
        self.univ_df = pd.read_csv(universal_player_csv)
        self.univ_df = self.univ_df.set_index('player_id')
        # Drop 'cricinfo_id' as it is redundant.
        self.univ_features = self.univ_df.drop(columns=['cricinfo_id'], errors='ignore')

        # Instead of a single file for player matches, we assume a directory where each player's
        # matches are stored in a file named '{player_id}.csv'. We build a list of valid player_ids
        # by checking which player match files exist and ensuring they have at least (context_len+1) rows.
        self.player_matches_dir = player_matches_dir
        self.player_ids = []
        self.player_match_data = {}  # Key: player_id, Value: DataFrame of that player's matches.
        for player_id in self.univ_features.index:
            match_file = os.path.join(player_matches_dir, f"{player_id}.csv")
            if os.path.exists(match_file):
                df_matches = pd.read_csv(match_file)
                # Assume the matches in this file are already sorted by date.
                if len(df_matches) >= (self.context_len + 1):
                    self.player_ids.append(player_id)
                    self.player_match_data[player_id] = df_matches

        self.match_players_dir = match_players_dir

        # Load the single match_info CSV and set match_id as index for fast lookup.
        self.match_info_df = pd.read_csv(match_info_csv)
        self.match_info_df = self.match_info_df.set_index('match_id')
        # Optionally drop columns not needed.
        self.match_info_df = self.match_info_df.drop(columns=['team1', 'team2', 'toss_winner', "toss_decision", "winner"], errors='ignore')
        #Todo don't just drop them we need to infer from them in sense of player's team ->winner 1 else 0

    def __len__(self):
        return len(self.player_ids)

    def __getitem__(self, idx):
        """
        Returns a dictionary with:
          - 'player_id': the player's id.
          - 'univ_features': universal features for the player.
          - 'context_matches': a numpy array of performance features for the context matches.
          - 'target_scores': a numpy array of target fantasy scores (for matches 2 ... context_len+1).
          - 'team1_players': list (per match) of team1 players' universal features.
          - 'team2_players': list (per match) of team2 players' universal features.
          - 'match_info': list (per match) of match info dictionaries.
        """
        #ToDO not just total_fantasy score, rather a weighted sum of error for batting_bowling fp's, fielding_fp with lesser weightage to the overall total_fp  
        # Select the player.
        player_id = self.player_ids[idx]
        # Retrieve the player's universal features as a numpy array.
        player_univ = self.univ_features.loc[player_id].values.astype(float)
        
        # Load this player's matches DataFrame from the pre-loaded dictionary.
        df_matches = self.player_match_data[player_id]
        total_matches = len(df_matches)
        # Randomly select a contiguous window of (context_len + 1) matches.
        start_idx = np.random.randint(1, total_matches - self.context_len+1)
        window_df = df_matches.iloc[start_idx-1 : start_idx + self.context_len]

        # For input, drop the unused columns.
        exclude_cols = ['teamname', 'match_id',
                        'strike_rate_fp', 'batting_fp', 'bowling_fp', 'fielding_fp', 'total_fp']
        # Extract target fantasy scores (from column 'total_fp') for matches 2 ... context_len+1.
        target_scores = window_df.iloc[1:self.context_len+1][['batting_fp', 'bowling_fp', 'fielding_fp',]].values.astype(float)
        #Todo account for all sub_fp's (stage -2)
        # Extract performance features for input matches.
        context_matches = window_df.iloc[:self.context_len].drop(columns=exclude_cols, errors='ignore')

        # For each match in the window, retrieve team players and match info.
        # team1_players_list = [] 
        team2_players_list = []
        match_info_list = []
        for _, match in window_df.iterrows():
            match_id = match['match_id']
            # Load match players file (expects a file named "{match_id}.csv").
            match_players_file = os.path.join(self.match_players_dir, f"{int(match_id)}.csv")
            match_players_df = pd.read_csv(match_players_file)
            
            # Determine the team of the sample's player in this match.
            player_team_series = match_players_df[
                match_players_df['player_id'] == player_id]['Team']
            #? rather we can refer it from window_df, there in the excluded_cols same as match['team_name']
            if not player_team_series.empty:
                player_team = player_team_series.iloc[0]
            else:
                player_team = match_players_df.iloc[0]['Team'] #?why's this, though it won't execute
            
            # Split player_ids into two groups based on the player's team.
            # team1_ids = match_players_df[
            #     match_players_df['Team'] == player_team]['player_id'].tolist()
            # #! we just need the team2 player_ids (we are omitting the intra-team interactions...)
            team2_ids = match_players_df[
                match_players_df['Team'] != player_team]['player_id'].tolist()

            # Retrieve universal features for these players.
            # team1_features = self.univ_features.reindex(team1_ids).dropna().values.astype(float)
            team2_features = self.univ_features.reindex(team2_ids).dropna().values.astype(float)
            # team1_players_list.append(team1_features)
            team2_players_list.append(team2_features)
            
            # Retrieve match info using match_id from the single match_info DataFrame.
            if match_id in self.match_info_df.index:
                match_info_dict = self.match_info_df.loc[match_id].to_dict()
            else:
                match_info_dict = { }
            match_info_list.append(match_info_dict)
            #? wht's this 

        # --- START OF CHANGES: convert to properly shaped torch.Tensors ---
        import torch

        # universal features
        univ_features = torch.tensor(player_univ, dtype=torch.float32)  # (feat_dim,)

        # context matches
        context_matches = torch.tensor(context_matches.values.astype(float),
                                       dtype=torch.float32)  # (context_len, perf_dim)

        # target scores
        target_scores = torch.tensor(target_scores, dtype=torch.float32)  # (context_len,)

        # team1 players: pad to max players across the window, then stack
        # max1 = max(arr.shape[0] for arr in team1_players_list)
        # feat_dim = team1_players_list[0].shape[1] if max1>0 else 0
        # padded1 = [
        #     np.pad(arr, ((0, max1 - arr.shape[0]), (0, 0)), mode='constant')
        #     for arr in team1_players_list
        # ]
        # team1_players = torch.tensor(np.stack(padded1), dtype=torch.float32)
        # # shape: (context_len+1, max1, feat_dim)

        # team2 players: same
        max2 = max(arr.shape[0] for arr in team2_players_list)
        feat_dim2 = team2_players_list[0].shape[1] if max2>0 else 0
        padded2 = [
            np.pad(arr, ((0, max2 - arr.shape[0]), (0, 0)), mode='constant')
            for arr in team2_players_list
        ]
        team2_players = torch.tensor(np.stack(padded2), dtype=torch.float32)
        # shape: (context_len+1, max2, feat_dim2)
        #? wht's this

        # match_info: convert list of dicts to array in fixed key order
        keys = list(self.match_info_df.columns)
        info_arr = np.stack([[d.get(k, 0.0) for k in keys] for d in match_info_list])
        match_info = torch.tensor(info_arr, dtype=torch.float32)
        # shape: (context_len+1, len(keys))

        # --- END OF CHANGES ---

        sample = {
            'player_id': player_id,
            'univ_features': univ_features,
            'context_matches': context_matches,
            'target_scores': target_scores,
            # 'team1_players': team1_players[:, :11, :],
            'team2_players': team2_players[:, :11, :],
            'match_info': match_info
        }
        
        if self.transform:
            sample = self.transform(sample)
        return sample


In [5]:
import os
from torch.utils.data import DataLoader

# Define file paths (update these paths as needed for your folder structure)
universal_player_csv = r'C:\Users\kumar\IPL_Fantasy_Score_Prediction\Ashu\Test_1\cleaned_universal_player.csv'
player_matches_dir = r'C:\Users\kumar\IPL_Fantasy_Score_Prediction\Ashu\Test_1\Processed_Player_records'  # Contains files named like {player_id}.csv (each containing that player's matches)
match_players_dir = r'C:\Users\kumar\IPL_Fantasy_Score_Prediction\Ashu\Test_1\processed_GlobalMatchrecords'    # Contains files like {match_id}.csv
match_info_csv = r'C:\Users\kumar\IPL_Fantasy_Score_Prediction\Ashu\Test_1\cleaned_matchinfo_without_venue_with_updated_match_number.csv'               # Single CSV containing all match info

# Define the context length (number of matches to use as context)
# For example, if config.CONTEXT_LEN is defined in your config module:
# config.CONTEXT_LEN = 5

# Initialize the dataset
dataset = PlayerMatchDataset(
    universal_player_csv=universal_player_csv,
    player_matches_dir=player_matches_dir,  # This parameter may be ignored if you use the directory version
    match_players_dir=match_players_dir,
    match_info_csv=match_info_csv,
    context_len=config.CONTEXT_LEN
)

# def get_shape(lst):
#     shape = []
#     while isinstance(lst, list):
#         shape.append(len(lst))
#         if len(lst) == 0:
#             break
#         lst = lst[0]
#     return tuple(shape)



# Initialize the DataLoader
# dataloader = DataLoader(dataset, batch_size=config.BATCH_SIZE, shuffle=True, num_workers=0)
len(dataset)



2354

In [6]:
dataloader = DataLoader(dataset, batch_size=config.BATCH_SIZE, shuffle=True, num_workers=0)
for batch in dataloader:
    print("Player IDs:", batch['player_id'])  # list of player_ids (length = batch_size)
    
    print("Universal features shape:", (batch['univ_features']).shape)  # e.g., (batch_size, num_features)
    
    print("Context matches shape:", batch['context_matches'].shape)  # e.g., (batch_size, context_len, performance_feature_dim)
    
    print("Target scores shape:", batch['target_scores'].shape)      # e.g., (batch_size, context_len)
    
    # The following are lists of length (context_len+1); each element is a numpy array.
    # team1_players_tensor = list_of_tensors_to_3d(batch['team1_players'])
    # team1_players_tensor = team1_players_tensor.squeeze(2)
    # print("Number of matches in team1_players (per sample):", batch['team1_players'].shape)
    
    # team2_players_tensor = list_of_tensors_to_3d(batch['team2_players'])
    # team2_players_tensor = team2_players_tensor.squeeze(2)
    print("Number of matches in team2_players (per sample):", batch['team2_players'].shape)
    
    # match_info_tensor = list_of_dicts_to_tensor(batch['match_info'])
    print("Number of matches in match_info (per sample):", (batch['match_info']).shape)
    break

Player IDs: ['3fca55af', '6cc4d322', '02ba80a0', '839b6d2e', 'f54c8804', '6c19c6e5', '8f6dd463', 'b23d3ec5', 'f7e9a562', 'afa7e784', 'fdb8ed0d', '92aeac25', 'f892fcf9', 'e741ed8f', '365229a8', '2494ff9e', '11f56b14', '7cb2d9d2', '13d0b3d6', '352e589f', 'ff3f6fc1', '23eeb873', '1be70c88', 'ffe699c0', '818c9e0d', '6a26221c', '9ccb6e52', 'f8035c2a', '773cec26', '9061a703', '7be8a0ed', '88209c84']
Universal features shape: torch.Size([32, 25])
Context matches shape: torch.Size([32, 16, 23])
Target scores shape: torch.Size([32, 16, 3])
Number of matches in team2_players (per sample): torch.Size([32, 17, 11, 25])
Number of matches in match_info (per sample): torch.Size([32, 17, 14])


In [7]:
def list_of_tensors_to_3d(tensor_list):
    """
    Converts a list of tensors into a 3D tensor with shape:
    (1, number_of_tensors, *inner_tensor_shape)
    
    Args:
        tensor_list (list of torch.Tensor): List of tensors with identical shapes.
        
    Returns:
        torch.Tensor: A tensor with the new shape (1, len(tensor_list), inner dims...).
    """
    # First stack the tensors along dimension 0: shape becomes (number_of_tensors, inner dims...)
    stacked = torch.stack(tensor_list, dim=0)
    # Add a leading dimension to obtain the final shape (1, number_of_tensors, inner dims...)
    return stacked.unsqueeze(0)


def list_of_dicts_to_tensor(data, key_order=None):
    """
    Convert a list of dictionaries (each with tensor or numeric values) 
    into a 3D tensor of shape (1, number_of_dicts, number_of_keys).

    Args:
        data (list): List of dictionaries where each dictionary contains the same keys.
        key_order (list, optional): Specific order of keys to extract from each dictionary.
                                    If None, keys from the first dictionary are used.

    Returns:
        torch.Tensor: A 3D tensor with shape (1, len(data), len(key_order)).
    """
    if not data:
        raise ValueError("The input data list is empty.")
    
    # Use keys from the first dictionary if no order is specified.
    if key_order is None:
        key_order = list(data[0].keys())
    
    values_list = []
    for d in data:
        # Extract values in the specified order. Convert tensor values to scalar if necessary.
        values = []
        for key in key_order:
            value = d[key]
            if isinstance(value, torch.Tensor):
                # Assuming tensor is of shape (1,)
                values.append(value.item())
            else:
                values.append(value)
        values_list.append(values)
    
    # Convert the list of lists to a 2D tensor.
    tensor_2d = torch.tensor(values_list)
    # Add a new dimension at the beginning to make it 3D.
    tensor_3d = tensor_2d.unsqueeze(0)
    return tensor_3d

## MODEL **Architechure**

In [76]:
class PlayerEmbedding(nn.Module):
  """ HEre we are doing the Proj of the raw Player embedding into the PERFORMANCE_EMBD_DIM """
  def __init__(self, in_channels=config.PLAYER_INPUT_DIM, out_channels=config.PERFORMANCE_EMBD_DIM):
    super().__init__()
    self.proj = nn.Linear(in_channels, out_channels)

  def forward(self, x):
    # print(f"In Player Embd  {x.shape}")
    # x: (B, PLAYER_INPUT_DIM) or flattened (B*T, PLAYER_INPUT_DIM)
    return self.proj(x)   ## ( B/B*T, PERFORMANCE_EMBD)

In [77]:
# class CrossAttention(nn.Module):
#     def __init__(self, d_embed=config.PERFORMANCE_EMBD_DIM, d_cross=config.PERFORMANCE_EMBD_DIM, n_heads=8, in_proj_bias=True, out_proj_bias=True):
#         super().__init__()
#         # Initialize linear layers for query, key, and value projections.
#         # q_proj: projects the query input with shape (batch_size, seq_length_q=1, d_embed)
#         self.q_proj   = nn.Linear(d_embed, d_embed, bias=in_proj_bias)
#         # k_proj: projects the key input with shape (batch_size, seq_length_kv=11, d_cross)
#         self.k_proj   = nn.Linear(d_cross, d_embed, bias=in_proj_bias)
#         # v_proj: projects the value input with shape (batch_size, seq_length_kv=11, d_cross)
#         self.v_proj   = nn.Linear(d_cross, d_embed, bias=in_proj_bias)
#         # out_proj: projects the concatenated multi-head outputs back to the original embedding dimension.
#         self.out_proj = nn.Linear(d_embed, d_embed, bias=out_proj_bias)
#         self.n_heads = n_heads                     # Number of attention heads.
#         self.d_head = d_embed // n_heads           # Dimensionality per head.

#     def forward(self, x, y):
#         """
#         Compute cross-attention between two sequences.

#         Parameters:
#             x (Tensor): Query input tensor of shape (batch_size, seq_length_q=1, d_embed)
#             y (Tensor): Key/Value input tensor of shape (batch_size, seq_length_kv=11, d_cross)

#         Returns:
#             Tensor: Output tensor after applying cross-attention, with shape (batch_size, seq_length_q=1, d_embed)
#         """
#         # Save original input shape.
#         input_shape = x.shape  # (batch_size, seq_length_q=1, d_embed)
#         batch_size, sequence_length, d_embed = input_shape

#         # Define intermediate shape for splitting into multiple heads.
#         # New shape: (batch_size, seq_length, n_heads, d_head)
#         interim_shape = (batch_size, -1, self.n_heads, self.d_head)

#         # Apply linear projections:
#         # q: project queries from x.
#         q = self.q_proj(x)  # (batch_size, seq_length_q=1, d_embed)
#         # k: project keys from y.
#         k = self.k_proj(y)  # (batch_size, seq_length_kv=11, d_embed)
#         # v: project values from y.
#         v = self.v_proj(y)  # (batch_size, seq_length_kv=11, d_embed)

#         # Reshape projections to separate attention heads.
#         # Transform shape to (batch_size, n_heads, seq_length, d_head)
#         q = q.view(interim_shape).transpose(1, 2)
#         k = k.view(interim_shape).transpose(1, 2)
#         v = v.view(interim_shape).transpose(1, 2)

#         # Compute attention scores using scaled dot-product:
#         # Multiply queries with transposed keys.
#         # Resulting shape: (batch_size, n_heads, seq_length_q=1, seq_length_kv=11)
#         weight = q @ k.transpose(-1, -2)

#         # Scale the scores by the square root of head dimension to stabilize gradients.
#         weight /= math.sqrt(self.d_head)

#         # Normalize the attention weights using softmax along the key dimension.
#         weight = F.softmax(weight, dim=-1)

#         # Compute the weighted sum over the values.
#         # Output shape: (batch_size, n_heads, seq_length_q=1, d_head)
#         output = weight @ v

#         # Reorder dimensions to combine multiple heads:
#         # Transpose back to (batch_size, seq_length_q=1, n_heads, d_head) and make contiguous.
#         output = output.transpose(1, 2).contiguous()

#         # Merge the multi-head outputs to recover original embedding dimension.
#         # Final shape: (batch_size, seq_length_q=1, d_embed)
#         output = output.view(input_shape)

#         # Apply the final output projection.
#         output = self.out_proj(output)

#         return output  # Return the final cross-attention output.

# given this cross attention module, i want to apply to calculate the cross attention from team2 to team1 players embeddings...
# #! handle cross attention (also should it be applied using team2 and current player)
# class MatchEmbedding(nn.Module):
#     """
#     Computes a match-level embedding from team and match information.
#     performing cross attention from team_players performance -> current palyer performance 
#     Expected input shapes (for T matches):
#       team1_players: (B, T, num_team1, PLAYER_INPUT_DIM) #!TODO we are passing it
#       team2_players: (B, T, num_team2, PLAYER_INPUT_DIM)
#       #? TODO implement cross attention (including player X team-2 performance )
      
#       match_info: (B, T, match_info_dim)
#     Output:
#       (B, T, PERFORMANCE_EMBD_DIM)
#     """
#     def __init__(self, player_embedding_module, in_channels=(2*config.PERFORMANCE_EMBD_DIM + config.MATCH_INPUT_EMBD), out_channels=config.PERFORMANCE_EMBD_DIM):
#         super().__init__()
#         self.player = player_embedding_module
#         self.proj = nn.Linear(in_channels, out_channels)

#     def forward(self, team1_players, team2_players, match_info):
#         """
#           1. Get player embeddings using self.player.
#           2. Sum (or pool) embeddings for each team.
#           3. Concatenate team representations with match_info.
#           4. Project the concatenated vector to obtain the final match embedding.
#         """
#         B, T, num_team1, _ = team1_players.shape
#         # print(f"In match embd {team1_players.shape} & {match_info.shape}, B {B}, T, {T}, nums_team1 {num_team1}")
#         # Compute player embeddings
#         team1_flat = team1_players.reshape(B * T, num_team1, -1)  # (B*T, num_team1, PLAYER_INPUT_DIM)
#         team1_embeds = self.player(team1_flat)  # (B*T, num_team1, PERFORMANCE_EMBD_DIM)
#         team1_sum = team1_embeds.sum(dim=1)  # (B*T, PERFORMANCE_EMBD_DIM)
#         team1_sum = team1_sum.reshape(B, T, -1)  # (B, T, PERFORMANCE_EMBD_DIM)

#         B, T, num_team2, _ = team2_players.shape
#         team2_flat = team2_players.reshape(B * T, num_team2, -1)
#         team2_embeds = self.player(team2_flat)  # (B*T, num_team2, PERFORMANCE_EMBD_DIM)
#         team2_sum = team2_embeds.sum(dim=1)  # (B*T, PERFORMANCE_EMBD_DIM)
#         team2_sum = team2_sum.reshape(B, T, -1)  # (B, T, PERFORMANCE_EMBD_DIM)

#         # Concatenate team summaries with match-level info along last dimension.
#         # match_info: (B, T, match_info_dim)
#         fused = torch.cat([team1_sum, team2_sum, match_info], dim=-1)  # (B, T, 2*PERFORMANCE_EMBD_DIM + match_info_dim)
#         match_embedding = self.proj(fused)  # (B, T, PERFORMANCE_EMBD_DIM)

#         return match_embedding 
# in this  MatchEmbedding module instead of duing a normal addition over all player in each team and creating team1_sum,team2_sum and then just concatenating them with match_info to create fused embedding of shape  (B, T, 2*PERFORMANCE_EMBD_DIM + match_info_dim)   




In [88]:
class CrossAttention(nn.Module):
    def __init__(self, d_embed=config.PERFORMANCE_EMBD_DIM, d_cross=config.PERFORMANCE_EMBD_DIM, n_heads=8, in_proj_bias=True, out_proj_bias=True):
        super().__init__()
        # Initialize linear layers for query, key, and value projections.
        # q_proj: projects the query input with shape (batch_size, seq_length_q=1, d_embed)
        self.q_proj   = nn.Linear(d_embed, d_embed, bias=in_proj_bias)
        # k_proj: projects the key input with shape (batch_size, seq_length_kv=11, d_cross)
        self.k_proj   = nn.Linear(d_cross, d_embed, bias=in_proj_bias)
        # v_proj: projects the value input with shape (batch_size, seq_length_kv=11, d_cross)
        self.v_proj   = nn.Linear(d_cross, d_embed, bias=in_proj_bias)
        # out_proj: projects the concatenated multi-head outputs back to the original embedding dimension.
        self.out_proj = nn.Linear(d_embed, d_embed, bias=out_proj_bias)
        self.n_heads = n_heads                     # Number of attention heads.
        self.d_head = d_embed // n_heads           # Dimensionality per head.

    def forward(self, x, y):
        """
        Compute cross-attention between two sequences.

        Parameters:
            x (Tensor): Query input tensor of shape (batch_size, seq_length_q=1, d_embed)
            y (Tensor): Key/Value input tensor of shape (batch_size, seq_length_kv=11, d_cross)

        Returns:
            Tensor: Output tensor after applying cross-attention, with shape (batch_size, seq_length_q=1, d_embed)
        """
        # Save original input shape.
        input_shape = x.shape  # (batch_size, seq_length_q=1, d_embed)
        batch_size, sequence_length, d_embed = input_shape

        # Define intermediate shape for splitting into multiple heads.
        # New shape: (batch_size, seq_length, n_heads, d_head)
        interim_shape = (batch_size, -1, self.n_heads, self.d_head)

        # Apply linear projections:
        # q: project queries from x.
        q = self.q_proj(x)  # (batch_size, seq_length_q=1, d_embed)
        # k: project keys from y.
        k = self.k_proj(y)  # (batch_size, seq_length_kv=11, d_embed)
        # v: project values from y.
        v = self.v_proj(y)  # (batch_size, seq_length_kv=11, d_embed)

        # Reshape projections to separate attention heads.
        # Transform shape to (batch_size, n_heads, seq_length, d_head)
        q = q.view(interim_shape).transpose(1, 2)
        k = k.view(interim_shape).transpose(1, 2)
        v = v.view(interim_shape).transpose(1, 2)

        # Compute attention scores using scaled dot-product:
        # Multiply queries with transposed keys.
        # Resulting shape: (batch_size, n_heads, seq_length_q=1, seq_length_kv=11)
        weight = q @ k.transpose(-1, -2)

        # Scale the scores by the square root of head dimension to stabilize gradients.
        weight /= math.sqrt(self.d_head)

        # Normalize the attention weights using softmax along the key dimension.
        weight = F.softmax(weight, dim=-1)

        # Compute the weighted sum over the values.
        # Output shape: (batch_size, n_heads, seq_length_q=1, d_head)
        output = weight @ v

        # Reorder dimensions to combine multiple heads:
        # Transpose back to (batch_size, seq_length_q=1, n_heads, d_head) and make contiguous.
        output = output.transpose(1, 2).contiguous()

        # Merge the multi-head outputs to recover original embedding dimension.
        # Final shape: (batch_size, seq_length_q=1, d_embed)
        output = output.view(input_shape)

        # Apply the final output projection.
        output = self.out_proj(output)

        return output  # Return the final cross-attention output.


In [89]:
class ApplyCrossAttention(nn.Module):
    """
        Computes a match-level embedding by refining a player's performance embedding 
        with cross-attention over the opposing team (team2) players' performance embeddings 
        and then fusing with match-level information. # we'll do this step later seperately
    """
    def __init__(self,cross_attention_module : CrossAttention,
                 in_channels = config.PERFORMANCE_EMBD_DIM + config.MATCH_INPUT_EMBD,
                 out_channels = config.PERFORMANCE_EMBD_DIM):
        super(ApplyCrossAttention,self).__init__()
        self.cross_attention = cross_attention_module
        self.proj = nn.Linear(in_channels,out_channels)
    def forward(self,player,team2_players,match_info):
        """
        expected inputed are already passed through the playerEmbedding module
        Args:
            player (Tensor): shape (B,T, PERFORMANCE_EMBD_DIM) #we have to copy it for (B,T,PLAYER_INPUT_DIM) already provided
            team2_players (Tensor): shape (B,T,num_team2,PERFORMANCE_EMBD_DIM)
            match_info (Tensor): shape (B,T,match_info_dim)
            
        Returns:
            Tensor: The final match embedding, with shape (B, T, PERFORMANCE_EMBD_DIM)
        """
        # print(player.shape)
        # print(team2_players.shape)
        assert player.shape[-1] == config.PERFORMANCE_EMBD_DIM, 'dim mismatch!...'
        B,T,num_team2,_ = team2_players.shape
        assert num_team2 == 11 , 'no of players in team2 should we 11...'
        # player_embed = self.player(player.unsqueeze(1))  # -> (B, 1, PERFORMANCE_EMBD_DIM)
        # player_embed_expanded = player_embed.expand(-1,T,-1)
        player_query = player.reshape(B*T,1,-1) #(B*T,1,PERFORMANCE_EMBD_DIM) 
        
        team2_embeds = team2_players.reshape(B*T,num_team2,-1) # -> (B*T, num_team2, PERFORMANCE_EMBD_DIM) 
        
        #TODO apply cross attention
        attened = self.cross_attention(player_query,team2_embeds)
        attened = attened.squeeze(1)  # -> (B*T, PERFORMANCE_EMBD_DIM)
        attened = attened.view(B,T,-1)
        
        fused = torch.cat([attened,match_info],dim=-1) # -> (B, T, PERFORMANCE_EMBD_DIM + match_info_dim) T is actually config.CONTEXT_LEN + 1
        match_embedding = self.proj(fused)  # -> (B, T, PERFORMANCE_EMBD_DIM)
        return match_embedding


In [90]:
class SelfAttention(nn.Module):
    def __init__(self, n_heads, d_embed, in_proj_bias=True, out_proj_bias=True):
        super().__init__()
        # This combines the Wq, Wk and Wv matrices into one matrix
        self.in_proj = nn.Linear(d_embed, 3 * d_embed, bias=in_proj_bias)
        # This one represents the Wo matrix
        self.out_proj = nn.Linear(d_embed, d_embed, bias=out_proj_bias)
        self.n_heads = n_heads   ## how many heads u want ?
        self.d_head = d_embed // n_heads   ## the original embedding get divided in the all heads equally
   

    def forward(self, x, causal_mask=False):

        # x: # (Batch_Size, Seq_Len, Dim)

        # (Batch_Size, Seq_Len, Dim)
        input_shape = x.shape

        # (Batch_Size, Seq_Len, Dim)
        batch_size, sequence_length, d_embed = input_shape

        # (Batch_Size, Seq_Len, H, Dim / H)
        qkv_shape = (batch_size, sequence_length, self.n_heads, self.d_head)

        # (Batch_Size, Seq_Len, Dim) -> (Batch_Size, Seq_Len, Dim * 3) -> 3 tensor of shape (Batch_Size, Seq_Len, Dim)
        q, k, v = self.in_proj(x).chunk(3, dim=-1)

        # (Batch_Size, Seq_Len, Dim) -> (Batch_Size, Seq_Len, H, Dim / H) -> (Batch_Size, H, Seq_Len, Dim / H)
        q = q.view(qkv_shape).transpose(1, 2)
        k = k.view(qkv_shape).transpose(1, 2)
        v = v.view(qkv_shape).transpose(1, 2)

        # (Batch_Size, H, Seq_Len, Dim / H) @ (Batch_Size, H, Dim / H, Seq_Len) -> (Batch_Size, H, Seq_Len, Seq_Len)
        weight = q @ k.transpose(-1, -2)

        if causal_mask:
            # Mask where the upper triangle (above the principal diagonal) is 1
            mask = torch.ones_like(weight, dtype=torch.bool).triu(1)
            # Fill the upper triangle with -inf
            weight.masked_fill_(mask, -torch.inf)

        # Divide by d_k (Dim / H).
        # (Batch_Size, H, Seq_Len, Seq_Len) -> (Batch_Size, H, Seq_Len, Seq_Len)
        weight /= math.sqrt(self.d_head)

        # (Batch_Size, H, Seq_Len, Seq_Len) -> (Batch_Size, H, Seq_Len, Seq_Len)
        weight = F.softmax(weight, dim=-1)

        # (Batch_Size, H, Seq_Len, Seq_Len) @ (Batch_Size, H, Seq_Len, Dim / H) -> (Batch_Size, H, Seq_Len, Dim / H)
        output = weight @ v

        # (Batch_Size, H, Seq_Len, Dim / H) -> (Batch_Size, Seq_Len, H, Dim / H)
        output = output.transpose(1, 2)

        # (Batch_Size, Seq_Len, H, Dim / H) -> (Batch_Size, Seq_Len, Dim)
        output = output.reshape(input_shape)

        # (Batch_Size, Seq_Len, Dim) -> (Batch_Size, Seq_Len, Dim)
        output = self.out_proj(output)

        # (Batch_Size, Seq_Len, Dim)
        return output


In [91]:
class a_layer(nn.Module):
    """ A Single Transformer Layer/Block """

    def __init__(self, n_head: int, n_embd: int):
        super().__init__()
        # Pre-attention norm
        self.layernorm_1 = nn.LayerNorm(n_embd)
        # Self attention
        self.attention = SelfAttention(n_head, n_embd)
        # Pre-FNN norm
        self.layernorm_2 = nn.LayerNorm(n_embd)
        # Feedforward layer
        self.linear_1 = nn.Linear(n_embd, 4 * n_embd)
        self.linear_2 = nn.Linear(4 * n_embd, n_embd)

    def forward(self, x):
        # (Batch_Size, Seq_Len, Dim)
        residue = x

        ### SELF ATTENTION ###

        # (Batch_Size, Seq_Len, Dim) -> (Batch_Size, Seq_Len, Dim)
        x = self.layernorm_1(x)

        # (Batch_Size, Seq_Len, Dim) -> (Batch_Size, Seq_Len, Dim)
        x = self.attention(x, causal_mask=True)

        # (Batch_Size, Seq_Len, Dim) + (Batch_Size, Seq_Len, Dim) -> (Batch_Size, Seq_Len, Dim)
        x += residue

        ### FEEDFORWARD LAYER ###
        # Apply a feedforward layer where the hidden dimension is 4 times the embedding dimension.

        residue = x
        # (Batch_Size, Seq_Len, Dim) -> (Batch_Size, Seq_Len, Dim)
        x = self.layernorm_2(x)

        # (Batch_Size, Seq_Len, Dim) -> (Batch_Size, Seq_Len, 4 * Dim)
        x = self.linear_1(x)

        # (Batch_Size, Seq_Len, 4 * Dim) -> (Batch_Size, Seq_Len, 4 * Dim)
        x = x * torch.sigmoid(1.702 * x)   # QuickGELU activation function found best for this work

        # (Batch_Size, Seq_Len, 4 * Dim) -> (Batch_Size, Seq_Len, Dim)
        x = self.linear_2(x)

        # (Batch_Size, Seq_Len, Dim) + (Batch_Size, Seq_Len, Dim) -> (Batch_Size, Seq_Len, Dim)
        x += residue

        return x

In [92]:

class NextFormPredictor1(nn.Module):
    """
    Predicts a sequence of performance embeddings autoregressively.
    applies loss at each time step. 
    """
    def __init__(self,player_embedding_module,cross_attention_module,cross_Attention_embedding_module,fantasy_score_prediction_module,custom_loss = None,embedding_dim = config.PERFORMANCE_EMBD_DIM ,num_layers=6, n_head=8):
        super(NextFormPredictor1,self).__init__()
        self.player_embedding_module = player_embedding_module
        self.cross_attention_module = cross_attention_module
        self.cross_attended_embedding = cross_Attention_embedding_module(self.cross_attention_module)
        
        self.performance_proj = nn.Linear(config.PERFORMANCE_INPUT_DIM, embedding_dim)
        # self.token_embedding = Performance_embedding_module(
        #     self.player_embedding_module,
        #     self.cross_attended_embedding
        # )
        
        self.pos_embedding = nn.Embedding(config.CONTEXT_LEN, embedding_dim)
        self.layers = nn.ModuleList([a_layer(n_head=n_head, n_embd=embedding_dim) for _ in range(num_layers)])
        self.layernorm = nn.LayerNorm(embedding_dim)
        self.out_proj = nn.Linear(embedding_dim, embedding_dim)
        self.fantasy_score_prediction_module = fantasy_score_prediction_module
        # self.custom_loss = custom_loss
        self.proj1 = nn.Linear(3*embedding_dim,embedding_dim)
        self.proj2 = nn.Linear(2*embedding_dim,embedding_dim)
    def forward(self,player_input,player_performance,team_2,match_info,target = None):
        """_summary_

        Args:
            player_input (Tensor): shape #(B,PLAYER_INPUT_DIM)
            player_performance (Tensor): (B,T + 1,PLAYER_PERPROMANCE_DIM)
            team_2 (Tensor): (B,T + 1,team2_size,PLAYER_INPUT_DIM)
            match_info (Tensor): (B,T + 1,match_info_dim)
            target (Tensor, optional): (B,T,_). Defaults to None.
        """
        B, T, _ = match_info.shape #correct it !...
        #! T is equal to config.CONTEXT_LEN + 1 
        assert T == config.CONTEXT_LEN + 1, 'context_len + 1 is not 17'
        player_embed = self.player_embedding_module(player_input) #(B,PERFORMANCE_EMBD_DIM)
        player_embed = player_embed.unsqueeze(1).repeat(1,T,1)   # (B,T, PERFORMANCE_EMBD_DIM)
        team_2_embed = self.player_embedding_module(team_2) #(B,T,team_2_size,PLAYER_INPUT_DIM) - > (B,T,team_2_size,PLAYER_EMBD_DIM)
        attended_embedding = self.cross_attended_embedding(player_embed,team_2_embed,match_info) # (B,T, PERFORMANCE_EMBD_DIM)
        
        player_performance_embd = self.performance_proj(player_performance.reshape(B*config.CONTEXT_LEN,-1)) # (B*T, PERFORMANCE_INPUT_DIM) => (B*T, PERFORMANCE_EMBD_DIM)
        player_performance_embd = player_performance_embd.reshape(B,Config.CONTEXT_LEN,-1) #(B,T,PERFORMANCE_EMBD_DIM) 
        fused = torch.cat([player_embed[:,:config.CONTEXT_LEN,:],attended_embedding[:,:config.CONTEXT_LEN,:],player_performance_embd[:,:config.CONTEXT_LEN,:]],dim=-1)
        # print(fused.shape)
        x = self.proj1(fused)
        pos_ids = torch.arange(config.CONTEXT_LEN, device=config.DEVICE).unsqueeze(0).expand(B,config.CONTEXT_LEN)  # (B, T)
        x = x + self.pos_embedding(pos_ids)
        for layer in self.layers:
            x = layer(x)
        x = self.layernorm(x)
        perf_emb = self.out_proj(x)  # (B, T, embedding_dim)
        # print("perf_emb: shape")
        # print(perf_emb.shape)
        pred_fantasy = None
        # loss = None
        if target is not None:
            pred_pref = perf_emb[:,:config.CONTEXT_LEN,:]  # (B, config.Context_len, embedding_dim)
            Bp,Tp,D = pred_pref.shape #!Tp ->config.CONTEXT_LEN
            # pred_pref = pred_pref.reshape(Bp * Tp, D)
            target_match_embd = torch.cat([player_embed[:,1:,:],attended_embedding[:,1:,:]]) # (B,T,2*PERFORMANCE_EMBD_DIM) attended_embd includes the match_info itself as cancatinated 
            target_match_embd = target_match_embd.view(Bp*Tp,-1)
            target_match_embd = self.proj2(target_match_embd) # (Bp,Tp,2*PERFORMANCE_EMBD_DIM) -> (Bp,Tp,PERFORMANCE_EMBD_DIM)
            target_match_embd = target_match_embd.reshape(Bp,Tp,-1)
            # print("target_match_embd: shape")
            # print(target_match_embd.shape)
            # print("player_embed_context_len: shape")
            # print(player_embed[:,1:,:].shape)
            
            pred_fantasy = self.fantasy_score_prediction_module(pred_pref,player_embed[:,1:,:],target_match_embd)  # we are back and forth giving player_embd till now we used it or passed it 3 times  
            #TODO need to handle if the shape is (B,T,3) 3 refers to batting_fp,bowling_fp,fielding_fp and total_fp is just some of these 
            
            # pred_fantasy_flat = pred_fantasy.view(-1,3) # (B,T,3) ->(B*T,)
            # target_flat = target.view(-1,3) #(B*T,)
            # loss = self.custom_loss(pred_fantasy,target)
            
        return pred_fantasy 
            
            
        

In [93]:
# experiment with the dropout layers
class FantasyScorePrediction(nn.Module):
    """Predict the Fantasy Score by the <CLS> embedding, the next match_embedding, and the player_embedding."""
    def __init__(self, embedding_dim=config.PERFORMANCE_EMBD_DIM):
        super().__init__()
        self.proj1 = nn.Linear(3 * config.PERFORMANCE_EMBD_DIM, 2048)
        self.proj2 = nn.Linear(2048, 512)
        self.proj3 = nn.Linear(512, 512)
        self.proj4 = nn.Linear(512, 256)
        self.proj5 = nn.Linear(256, 256)
        self.proj6 = nn.Linear(256, 128)
        self.proj7 = nn.Linear(128, 128)
        self.proj8 = nn.Linear(128, 3)

    def forward(self, pred_pref, target_player_embd, target_match_embd): 
        # Concatenate the embeddings along the last dimension
        # print(f"NI fantasy {pred_pref.shape} & {target_player_embd.shape} & {target_match_embd.shape}")
        x = torch.cat([pred_pref, target_player_embd, target_match_embd], dim=-1)
        # print(f"/n x.shape ->{x.shape} ")
        x = F.gelu(self.proj1(x))
        x = F.gelu(self.proj2(x))
        x = F.gelu(self.proj3(x))
        x = F.gelu(self.proj4(x))
        x = F.gelu(self.proj5(x))
        x = F.gelu(self.proj6(x))
        x = F.gelu(self.proj7(x))


        x = self.proj8(x)

        return x


In [94]:

#TODO considering equal weightage to each time step...
def custom_loss_component_fps(pred_fantasy: torch.Tensor,target_components: torch.Tensor,target_total: torch.Tensor = None,gamma : float = 0.5,weights :torch.Tensor= None) -> torch.Tensor:
    """
    computes a weighted loss for multi-component fantasy score prediction.
    By including a term that directly compares the sum of the predictions with the
    overall target (i.e. total_fp), you ensure that the model learns to get the “big picture” right. 
    At the same time, the separate losses help the model capture the underlying 
    structure: where batting and bowling (which are supported by richer
    historical data and have more influence) get more weight than fielding.
    Args:
        pred_fantasy (torch.Tensor): Model prediction of shape (B,T,3), where
                        pred_fantasy[:,:,0] -> bating_fp,
                        pred_fantasy[:,:1] -> bowling_fp,
                        pred_fantasy[:,:2] -> fielding_fp...
        target_components (torch.Tensor): Ground truth for the components, shape (B,T,3), with same ordering
        target_total (torch.Tensor,optional) : Ground truth total fantasy points of shape (B,T).
                                               If not provided, it is computed as sum(target_components, dim=1).
        gamma (float,optional) : Trade-off hyperparameter between total and component losses.
                                value E (0,1) and default is 0.5.
        weights (torch.Tensor,optional) :  weights (torch.Tensor, optional): Weights to be applied to each time step's loss.
            Expected shape is (T,). If None, each time step is equally weighted.
    Returns : 
    total_loss (torch.Tensor) : Scalar loss combining total and component-level errors.

    """
    B,T,_ = pred_fantasy.shape
    pred_total = pred_fantasy.sum(dim = -1) #(B,T)
    
    if target_total is None:
        target_total = target_components.sum(dim = -1) #(B,T)
    abs_error_total = torch.abs(pred_total - target_total) #(B,T)
    loss_total = (pred_total - target_total)**2 #(B,T)
    comp_weights = torch.tensor([0.4,0.4,0.2],device=pred_fantasy.device, dtype = pred_fantasy.dtype)
    loss_components = (((pred_fantasy - target_components)**2)*comp_weights).sum(dim=-1)
    combined_loss = gamma * loss_total + (1 - gamma)*loss_components       #(B,T) 
    if weights is not  None:        
        # weights is expected to have shape (T,)

        assert weights.dim() == 1 and weights.shape[0] == T, "weights must be a 1D tensor of shape (T,)"
        combined_loss = combined_loss*weights   
    total_loss = combined_loss.mean()
    return total_loss
        

    
    
    

In [95]:
def custom_asymmetric_loss(pred, target, delta=30.0, penalty=2.0, extra_factor=1.5):
    """
    Asymmetric Huber-style loss that emphasizes errors above 10.
    
    Parameters:
        pred (Tensor): Model predictions (floating point)
        target (Tensor): Target values (integers)
        delta (float): Transition point from quadratic to linear penalty.
        penalty (float): Factor to amplify the loss when pred < target.
        extra_factor (float): Additional factor to scale the loss in the linear region, if desired.
    
    Returns:
        Tensor: Mean loss value.
    """
    error = pred - target
    abs_err = error.abs()
    
    # Compute the quadratic (small error) region
    loss_small = torch.where(
        error < 0,
        penalty * 0.5 * error.pow(2),
        0.5 * error.pow(2)
    )
    
    # Compute the linear (large error) region with optional extra scaling
    base_loss_large = torch.where(
        error < 0,
        penalty * (delta * (abs_err - 0.5 * delta)),
        delta * (abs_err - 0.5 * delta)
    )
    
    # Optionally apply an extra factor to errors that are above the desired threshold
    loss_large = torch.where(
        abs_err > delta,
        extra_factor * base_loss_large,
        base_loss_large
    )

    # for err in abs_err:
    #     if err <= delta:
    #         print("yEs")
    #     else:
    #         print("Np")

    loss = torch.where(abs_err <= delta, loss_small, loss_large)
    # print(abs_err.mean())
    return loss.mean()


In [96]:

#TODO considering equal weightage to each time step...
def custom_loss_component_fps_accounting_for_outliers(
    pred_fantasy: torch.Tensor,
    target_components: torch.Tensor,
    target_total: torch.Tensor = None,
    gamma : float = 0.5,weights :torch.Tensor= None,
    epoch: int = 0,
    total_epoches : int = 100,
    decay_rate : float = 0.05, # decides how fast the threshold converges..
    upper_threshold : float = 200,
    min_threshold : float = 100,
    
    ) -> torch.Tensor:
    """
    computes a weighted loss for multi-component fantasy score prediction.
    By including a term that directly compares the sum of the predictions with the
    overall target (i.e. total_fp), you ensure that the model learns to get the “big picture” right. 
    At the same time, the separate losses help the model capture the underlying 
    structure: where batting and bowling (which are supported by richer
    historical data and have more influence) get more weight than fielding.
    Args:
        pred_fantasy (torch.Tensor): Model prediction of shape (B,T,3), where
                        pred_fantasy[:,:,0] -> bating_fp,
                        pred_fantasy[:,:1] -> bowling_fp,
                        pred_fantasy[:,:2] -> fielding_fp...
        target_components (torch.Tensor): Ground truth for the components, shape (B,T,3), with same ordering
        target_total (torch.Tensor,optional) : Ground truth total fantasy points of shape (B,T).
                                               If not provided, it is computed as sum(target_components, dim=1).
        gamma (float,optional) : Trade-off hyperparameter between total and component losses.
                                value E (0,1) and default is 0.5.
        weights (torch.Tensor,optional) :  weights (torch.Tensor, optional): Weights to be applied to each time step's loss.
            Expected shape is (T,). If None, each time step is equally weighted.
    Returns : 
    total_loss (torch.Tensor) : Scalar loss combining total and component-level errors.

    """
    B,T,_ = pred_fantasy.shape
    pred_total = pred_fantasy.sum(dim = -1) #(B,T)
    
    if target_total is None:
        target_total = target_components.sum(dim = -1) #(B,T)
    abs_error_total = torch.abs(pred_total - target_total) #(B,T)
    # effective_threshold = math.exp(-decay_rate*epoch) # it should start as inf(consider,upper_threshold = 250) and gradualy decrease toward min_threshold as epoch -> total_epoch
    effective_threshold = (upper_threshold - min_threshold) * math.exp(-decay_rate * epoch) + min_threshold
    # For errors that exceed the threshold reduce their panelity 
    # otherwise keep their weighting at 1.
    weight_total = torch.where(
        abs_error_total > effective_threshold,
        torch.exp(-decay_rate * (abs_error_total - effective_threshold)),
        torch.ones_like(abs_error_total)        
    )
    loss_total = (pred_total - target_total)**2 #(B,T)
    comp_weights = torch.tensor([0.4,0.4,0.2],device=pred_fantasy.device, dtype = pred_fantasy.dtype)
    loss_components = (((pred_fantasy - target_components)**2)*comp_weights).sum(dim=-1)
    combined_loss = gamma * loss_total + (1 - gamma)*loss_components       #(B,T) 
    combined_loss = combined_loss*weight_total
    if weights is not  None:        
        # weights is expected to have shape (T,)

        assert weights.dim() == 1 and weights.shape[0] == T, "weights must be a 1D tensor of shape (T,)"
        combined_loss = combined_loss*weights   
    total_loss = combined_loss.mean()
    return total_loss
        

    
    
    

In [97]:
player_embedding_module = PlayerEmbedding(
    in_channels=config.PLAYER_INPUT_DIM,
    out_channels=config.PERFORMANCE_EMBD_DIM
)

# Note: NextFormPredictor expects the *class* MatchEmbedding so that inside it
# it can do `match_embedding_module(player_embedding_module)`
cross_attention_module = CrossAttention()
cross_attention_embedding_module = ApplyCrossAttention
fantasy_score_prediction_module = FantasyScorePrediction(
    embedding_dim=config.PERFORMANCE_EMBD_DIM
)
custom_loss = custom_loss_component_fps_accounting_for_outliers

model = NextFormPredictor1(
    player_embedding_module=player_embedding_module,
    cross_attention_module= cross_attention_module,
    cross_Attention_embedding_module=cross_attention_embedding_module,
    fantasy_score_prediction_module=fantasy_score_prediction_module,
    # custom_loss=custom_loss,
    embedding_dim=config.PERFORMANCE_EMBD_DIM,
    num_layers=6,
    n_head=8
).to(config.DEVICE)



optimizer = torch.optim.AdamW(model.parameters(), lr=config.LEARNING_RATE)


In [98]:
from tqdm import tqdm
from typing import List

def train(model, dataloader, start_epoch=0) -> List[torch.Tensor]:
    model.to(config.DEVICE)
    optimizer = torch.optim.AdamW(model.parameters(), lr=config.LEARNING_RATE)
    epoch_losses = []

    for epoch in range(start_epoch, config.NUM_EPOCHS):
        model.train()
        total_loss = 0.0

        # Wrap dataloader with tqdm for progress bar
        loop = tqdm(dataloader, leave=True)
        loop.set_description(f"Epoch [{epoch+1}/{config.NUM_EPOCHS}]")

        for batch in loop:
            # Move tensors to device
            for k, v in batch.items():
                if torch.is_tensor(v):
                    batch[k] = v.to(config.DEVICE)

            # Unpack batch
            player_univ   = batch['univ_features']
            context_feats = batch['context_matches']
            # team1_players = batch['team1_players']
            team2_players = batch['team2_players']
            match_info    = batch['match_info']
            target_scores = batch['target_scores']
            
            # print(player_univ.shape)
            # print(context_feats.shape)
            # print(team1_players.shape)
            # print(team2_players.shape)
            # print(match_info.shape)
            # print(target_scores.shape)

            optimizer.zero_grad()
            pred_fantasy = model(
                player_univ,
                context_feats,
                # team1_players,
                team2_players,
                match_info,
                target_scores
            )
            loss = custom_loss(pred_fantasy,target_scores,epoch=epoch,total_epoches=config.NUM_EPOCHS,min_threshold=50)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

            # Update tqdm progress bar with current loss
            loop.set_postfix(loss=loss.item())

        avg_loss = total_loss / len(dataloader)
        epoch_losses.append(avg_loss)

        print(f"Epoch {epoch+1}/{config.NUM_EPOCHS} — Avg Loss: {avg_loss:.4f}")

    return epoch_losses


In [ ]:
lossi = train(model, dataloader)


Epoch [1/100]: 100%|██████████| 74/74 [03:49<00:00,  3.10s/it, loss=964]    


Epoch 1/100 — Avg Loss: 1319.7701


Epoch [2/100]:  20%|██        | 15/74 [01:01<03:11,  3.25s/it, loss=1.23e+3]